In [1]:
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import plotnine as p9
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# To analyze the correlation between pA and rM soRNA-seq.
# Align the ERCC normalized raw counts of each gene in each method. 
# Output: the dot plot of raw counts in pA and in rM, in ctr and in cKO

In [3]:
# solo rM-seq, yg samples raw counts
df_rM_GV = pd.read_csv("../raw/RNAseq/rM_rawCounts.csv",sep=' ')
df_rM_GV

# only get ERCC:
df_rM_GV_ERCC = df_rM_GV[df_rM_GV.index.str.startswith('ERCC')]
df_rM_GV_ERCC

# only get genes without ERCC:
df_rM_GV = df_rM_GV[df_rM_GV.index.str.startswith('ENSM')]
df_rM_GV


# each count normalized by ERCC.mean:
df_rM_GV_norm = df_rM_GV.div(df_rM_GV_ERCC.mean(axis=0),axis=1)
df_rM_GV_norm

# get rowMeans of ctr and cKO:
df_rM_GV_norm['ctr_means'] = df_rM_GV_norm.loc[:,df_rM_GV_norm.columns.str.startswith('dis3l2_yg_ctr')].sum(axis=1)
df_rM_GV_norm['cKO_means'] = df_rM_GV_norm.loc[:,df_rM_GV_norm.columns.str.startswith('dis3l2_yg_cKO')].sum(axis=1)

df_rM_GV_norm_simple = df_rM_GV_norm[['ctr_means','cKO_means']]
df_rM_GV_norm_simple


,ctr_means,cKO_means
ENSMUSG00000000001,56.259849,56.178530
ENSMUSG00000000003,0.000000,0.000000
ENSMUSG00000000028,28.425895,31.593488
ENSMUSG00000000031,0.022196,0.009857
ENSMUSG00000000037,117.991650,88.138694
...,...,...
ENSMUSG00000118655,0.000000,0.000000
ENSMUSG00000118656,0.000000,0.000000
ENSMUSG00000118657,0.000000,0.000000
ENSMUSG00000118658,0.000000,0.000000


In [5]:
# pA-seq, yg samples raw counts
df_pA_GV = pd.read_csv("../raw/RNAseq/pA_rawCounts_gene.csv",sep=' ')
df_pA_GV

# only get genes at GV stage
df_pA_GV = df_pA_GV.loc[:,df_pA_GV.columns.str.startswith('dis3l2GV')]
df_pA_GV.columns = df_pA_GV.columns.str.replace('_filtered_genes_no.txt','')

# only get ERCC:
df_pA_GV_ERCC = pd.read_csv("../raw/RNAseq/pA_rawCounts_ERCC.csv",sep=' ')
df_pA_GV_ERCC

# only get genes at GV stage
df_pA_GV_ERCC = df_pA_GV_ERCC.loc[:,df_pA_GV_ERCC.columns.str.startswith('dis3l2GV')]
df_pA_GV_ERCC.columns = df_pA_GV_ERCC.columns.str.replace('_ERCC_count.txt','')


# each count normalized by ERCC.mean:
df_pA_GV_norm = df_pA_GV.div(df_pA_GV_ERCC.mean(axis=0),axis=1)
df_pA_GV_norm


# get rowMeans of ctr and cKO:
df_pA_GV_norm['ctr_means'] = df_pA_GV_norm.loc[:,df_pA_GV_norm.columns.str.startswith('dis3l2GV_ctr')].sum(axis=1)
df_pA_GV_norm['cKO_means'] = df_pA_GV_norm.loc[:,df_pA_GV_norm.columns.str.startswith('dis3l2GV_cKO')].sum(axis=1)

df_pA_GV_norm_simple = df_pA_GV_norm[['ctr_means','cKO_means']]
df_pA_GV_norm_simple

,ctr_means,cKO_means
ENSMUSG00000000001,33.405648,27.191902
ENSMUSG00000000003,0.000000,0.000000
ENSMUSG00000000028,7.344158,6.984271
ENSMUSG00000000031,0.000000,0.022479
ENSMUSG00000000037,23.222680,8.567202
...,...,...
ENSMUSG00000118655,0.003032,0.000000
ENSMUSG00000118656,0.000000,0.000000
ENSMUSG00000118657,0.000000,0.000000
ENSMUSG00000118658,0.000000,0.000000


In [6]:
# merge two df:
df_rM_pA_GV = pd.merge(left=df_rM_GV_norm_simple, right=df_pA_GV_norm_simple, 
                       left_on=df_rM_GV_norm_simple.index,right_on=df_pA_GV_norm_simple.index)
df_rM_pA_GV
df_rM_pA_GV.columns = ['id','ctr_rM','cKO_rM','ctr_pA','cKO_pA']
df_rM_pA_GV = df_rM_pA_GV.loc[:, ['ctr_rM','cKO_rM','ctr_pA','cKO_pA']]

# add 1 and log10
df_rM_pA_GV
df_rM_pA_GV = log10(df_rM_pA_GV+1)
df_rM_pA_GV

,ctr_rM,cKO_rM,ctr_pA,cKO_pA
0,1.757850,1.757233,1.536630,1.450124
1,0.000000,0.000000,0.000000,0.000000
2,1.468730,1.513131,0.921383,0.902235
3,0.009534,0.004260,0.000000,0.009654
4,2.075516,1.950066,1.384222,0.980785
...,...,...,...,...
55482,0.000000,0.000000,0.001315,0.000000
55483,0.000000,0.000000,0.000000,0.000000
55484,0.000000,0.000000,0.000000,0.000000
55485,0.000000,0.000000,0.000000,0.000000


In [7]:
# need define intersect value. now sure using log10.
df_rM_pA_GV['ctr_diff'] = df_rM_pA_GV['ctr_rM']-df_rM_pA_GV['ctr_pA']
df_rM_pA_GV['ctr_diff'].abs().describe(percentiles=[0.95])

count    55487.000000
mean         0.075138
std          0.163141
min          0.000000
50%          0.001637
95%          0.459982
max          1.803235
Name: ctr_diff, dtype: float64

In [17]:
# get the Ctrl group, ribominus-biased genes:
df_rM_pA_GV['cKO_diff'] = df_rM_pA_GV['cKO_rM']-df_rM_pA_GV['cKO_pA']
df_rM_pA_GV[df_rM_pA_GV['cKO_diff'] >0.46].count()

ctr_rM      5237
cKO_rM      5237
ctr_pA      5237
cKO_pA      5237
ctr_diff    5237
cKO_diff    5237
dtype: int64

In [9]:
55487-2177-596

52714

In [10]:
52714/55487

0.9500243300232487

In [16]:
# plot df_rM_pA_GV
intercept = 0.459982
plot = (
    p9.ggplot()
    + p9.geom_point(df_rM_pA_GV, 
                  p9.aes(x='ctr_pA', y='ctr_rM'), size=2,alpha=0.02)
#     + p9.geom_point(rM_specific, 
#                   p9.aes(x='log2FC_pA', y='log2FC_rM'), size=2,alpha=0.05,color = 'magenta')
#     + p9.geom_point(pA_specific, 
#                   p9.aes(x='log2FC_pA', y='log2FC_rM'), size=2,alpha=0.05,color = 'blue')
    
#     + p9.scale_color_manual(breaks = 'group',
#                            values = ['blue','magenta','gray'])

    + p9.geom_abline(intercept = 0, slope = 1, linetype='dashed',color='gray',size=1)
    + p9.geom_abline(intercept = intercept, slope = 1, linetype='dashed',color='red',size=2)
    + p9.geom_abline(intercept = -intercept, slope = 1, linetype='dashed',color='blue',size=2)
    + p9.geom_vline(xintercept = 0, linetype='dashed')
    + p9.geom_hline(yintercept = 0, linetype='dashed')
    + p9.theme(void)
#     + p9.facet_wrap("genotype")
#     + p9.labs(title = 'ctr')
    + p9.coord_fixed(ratio=1,xlim=(0,4),ylim=(0,4))
)
plot
plot.save("../results/pA_vs_rM_rawCounts_" + 'ctr_rMvspA'+ ".tiff", height=4, width=4)


/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 4 x 4 in image.
  from_inches(height, units), units), PlotnineWarning)
/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: ../results/pA_vs_rM_rawCounts_ctr_rMvspA.tiff
  warn('Filename: {}'.format(filename), PlotnineWarning)


In [15]:
# plot df_rM_pA_GV
plot = (
    p9.ggplot()
    + p9.geom_point(df_rM_pA_GV, 
                  p9.aes(x='cKO_pA', y='cKO_rM'), size=2,alpha=0.02)
#     + p9.geom_point(rM_specific, 
#                   p9.aes(x='log2FC_pA', y='log2FC_rM'), size=2,alpha=0.05,color = 'magenta')
#     + p9.geom_point(pA_specific, 
#                   p9.aes(x='log2FC_pA', y='log2FC_rM'), size=2,alpha=0.05,color = 'blue')
    
#     + p9.scale_color_manual(breaks = 'group',
#                            values = ['blue','magenta','gray'])

    + p9.geom_abline(intercept = 0, slope = 1, linetype='dashed',color='gray',size=1)
    + p9.geom_abline(intercept = intercept, slope = 1, linetype='dashed',color='red',size=2)
    + p9.geom_abline(intercept = -intercept, slope = 1, linetype='dashed',color='blue',size=2)
    + p9.geom_vline(xintercept = 0, linetype='dashed')
    + p9.geom_hline(yintercept = 0, linetype='dashed')
    + p9.theme(void)
#     + p9.facet_wrap("genotype")
#     + p9.labs(title = 'cKO')
    + p9.coord_fixed(ratio=1,xlim=(0,4),ylim=(0,4))
)
plot
plot.save("../results/pA_vs_rM_rawCounts_" + 'cKO_rMvspA'+ ".tiff", height=4, width=4)


/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:729: PlotnineWarning: Saving 4 x 4 in image.
  from_inches(height, units), units), PlotnineWarning)
/opt/anaconda3/lib/python3.7/site-packages/plotnine/ggplot.py:730: PlotnineWarning: Filename: ../results/pA_vs_rM_rawCounts_cKO_rMvspA.tiff
  warn('Filename: {}'.format(filename), PlotnineWarning)


In [13]:
# write to csv:
df_rM_pA_GV_short.to_csv("../results/metadata/df_rM_pA_GV_short.csv")